In [2]:
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from nltk import word_tokenize
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
import os
import chardet
import pandas as pd
import numpy as np
import nltk
import string
import copy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [42]:

df = pd.read_csv("C:\\TextMining\\QAquiz\\input\\WebQA.csv")
df = df.dropna(how="any").reset_index(drop=True)

In [43]:
print(df.shape)
print(df.columns)

(9, 3)
Index(['qid', 'question', 'answer'], dtype='object')


In [44]:
a = 0 
for i in range(a,a+9):
    print(df.question[i])
    print(df.answer[i])
    print()

What are Web Services?
Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.

What are the different types of web services?
There are two types of web services 

SOAP   It is an XML-based protocol for accessing web services.
RESTful  It is an architectural style, not a protocol.

What are the main features of SOAP?
SOAP is a communication protocol.
SOAP communicates between applications.
SOAP is a format for sending messages.
SOAP is designed to communicate via Internet.
SOAP is platform independent.
SOAP is language independent.
SOAP is simple and extensible.
SOAP allows you to get around firewalls.
SOAP developed as a W3C standard

What is RESTful web services?
The REST stands for Representational State Transfer. It is an architectural style. It is not a protocol like SOAP

What is SOA?
SOA stands for Service Oriented Architecture. It is a de

In [45]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop_words = stopwords.words('english')

In [62]:
def preprocess(pd):
  pd = pd.str.lower()
  pd = pd.str.replace('[{}]'.format(string.punctuation), ' ')
  pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
  pd = pd.apply(lambda x: [item for item in x if item not in stop_words])
  #pd = pd.apply(lambda x: [stemmer.stem(y) for y in x])
  return pd.str.join(' ')

   # Function to convert  list to str 
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

In [47]:
df['o_len_q'] = df.question.apply(lambda x: len(str(x)))
df['o_len_word_q'] = df.question.apply(lambda x: len(str(x).split()))
df['o_len_a'] = df.answer.apply(lambda x: len(str(x)))
df['o_len_word_a'] = df.answer.apply(lambda x: len(str(x).split()))
df['o_processed_ans']=preprocess(df['answer'])
df['o_plen_a'] = df.o_processed_ans.apply(lambda x: len(str(x)))
df['o_plen_word_a'] = df.o_processed_ans.apply(lambda x: len(str(x).split()))

In [48]:
df.head(2)

,qid,question,answer,o_len_q,o_len_word_q,o_len_a,o_len_word_a,o_processed_ans,o_plen_a,o_plen_word_a
0,1,What are Web Services?,Web services are XML based information exchang...,22,4,197,25,web service xml based information exchange sys...,152,19
1,2,What are the different types of web services?,There are two types of web services \n\nSOAP ...,45,8,155,26,two type web service soap xml based protocol a...,103,15


In [26]:
question = input("Enter question? ")
Answer_R= input("Enter Answer ")
#df1 = {[question],[Answer_R]}
df1 = {
    'ques':[question],
    'Answer_R':[Answer_R]}
 
ques_answer=pd.DataFrame(df1,columns=['ques','Answer_R'])


Enter question? What are Web Services?
Enter Answer Web services are XML based information exchange systems that use the Internet for direct application-to-application 


In [27]:
ques_answer.head(2)


,ques,Answer_R
0,What are Web Services?,Web services are XML based information exchang...


In [49]:
ques_answer['len_a'] = ques_answer.Answer_R.apply(lambda x: len(str(x)))
ques_answer['len_word_a'] = ques_answer.Answer_R.apply(lambda x: len(str(x).split()))
ques_answer['processed_ans']=preprocess(ques_answer['Answer_R'])
ques_answer['plen_a'] = ques_answer.processed_ans.apply(lambda x: len(str(x)))
ques_answer['plen_word_a'] = ques_answer.processed_ans.apply(lambda x: len(str(x).split()))

In [52]:
#Fetch Answer from the corpous

o_ques_answer = df[df['question'].isin(ques_answer['ques'])]

In [53]:
o_ques_answer.head(2)

,qid,question,answer,o_len_q,o_len_word_q,o_len_a,o_len_word_a,o_processed_ans,o_plen_a,o_plen_word_a
0,1,What are Web Services?,Web services are XML based information exchang...,22,4,197,25,web service xml based information exchange sys...,152,19


In [58]:
o_ans=o_ques_answer['o_processed_ans'].tolist()
ans=ques_answer['processed_ans'].tolist()


In [60]:
print(o_ans)
print(ans)

['web service xml based information exchange system use internet direct application application interaction system include program object message document']
['web service xml based information exchange system use internet direct application application']


In [63]:
original_answer = nltk.word_tokenize(listToString(o_ans))
answered = nltk.word_tokenize(listToString(ans))

In [64]:
print(original_answer)
print(answered)

['web', 'service', 'xml', 'based', 'information', 'exchange', 'system', 'use', 'internet', 'direct', 'application', 'application', 'interaction', 'system', 'include', 'program', 'object', 'message', 'document']
['web', 'service', 'xml', 'based', 'information', 'exchange', 'system', 'use', 'internet', 'direct', 'application', 'application']


In [27]:
print(dictionary)

Dictionary(25 unique tokens: ['current', 'f1', 'international', 'master’s', 'mean']...)


In [32]:
import gensim.downloader as api

w2v_model = api.load("glove-wiki-gigaword-50")
similarity_matrix = w2v_model.similarity_matrix(dictionary)
#print('similarity = %.4f' % similarity_matrix)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity_matrix` (Method will be removed in 4.0.0, use gensim.models.keyedvectors.WordEmbeddingSimilarityIndex instead).
  after removing the cwd from sys.path.


In [26]:
from gensim.matutils import softcossim

similarity = softcossim(question1, question2, similarity_matrix)
print('similarity = %.4f' % similarity)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `softcossim` (Function will be removed in 4.0.0, use gensim.similarities.termsim.SparseTermSimilarityMatrix.inner_product instead).
  This is separate from the ipykernel package so we can avoid doing imports until


similarity = 0.4756


In [30]:
df['len_q1'] = df.question1.apply(lambda x: len(str(x)))
df['len_q2'] = df.question2.apply(lambda x: len(str(x)))
df['diff_len'] = df.len_q1 - df.len_q2
df['len_char_q1'] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
df['len_char_q2'] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
df['len_word_q1'] = df.question1.apply(lambda x: len(str(x).split()))
df['len_word_q2'] = df.question2.apply(lambda x: len(str(x).split()))
df['common_words'] = df.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)

In [31]:

df.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,66,57,9,20,20,14,12,10
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,51,88,-37,21,29,8,13,4


In [47]:


news_headline1 = "Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents."
news_headline2 = "Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents."
news_headline3 = "Web services are XML based information  systems and use Internet for application interaction.They contain programs, objects, messages, or documents."
news_headline4 = "Web services Web services Web services XML Web services XML Internet "

news_headlines = [news_headline1, news_headline2, news_headline3, news_headline4]

In [48]:
news_headline1_tokens = nltk.word_tokenize(news_headline1)
news_headline2_tokens = nltk.word_tokenize(news_headline2)
news_headline3_tokens = nltk.word_tokenize(news_headline3)
news_headline4_tokens = nltk.word_tokenize(news_headline4)

for words in [news_headline1_tokens, news_headline2_tokens, news_headline3_tokens, news_headline4_tokens]:
    print('First 10 tokens from news headlines: ', words[:10])

First 10 tokens from news headlines:  ['Web', 'services', 'are', 'XML', 'based', 'information', 'exchange', 'systems', 'that', 'use']
First 10 tokens from news headlines:  ['Web', 'services', 'are', 'XML', 'based', 'information', 'exchange', 'systems', 'that', 'use']
First 10 tokens from news headlines:  ['Web', 'services', 'are', 'XML', 'based', 'information', 'systems', 'and', 'use', 'Internet']
First 10 tokens from news headlines:  ['Web', 'services', 'Web', 'services', 'Web', 'services', 'XML', 'Web', 'services', 'XML']


In [49]:
from numpy import argmax

def transform(headlines):
    tokens = [w for s in headlines for w in s ]
    print()
    print('All Tokens:')
    print(tokens)

    results = []
    label_enc = sklearn.preprocessing.LabelEncoder()
    onehot_enc = sklearn.preprocessing.OneHotEncoder()
    
    encoded_all_tokens = label_enc.fit_transform(list(set(tokens)))
    encoded_all_tokens = encoded_all_tokens.reshape(len(encoded_all_tokens), 1)
    
    onehot_enc.fit(encoded_all_tokens)
    
    for headline_tokens in headlines:
        print()
        print('Original Input:', headline_tokens)
        
        encoded_words = label_enc.transform(headline_tokens)
        print('Encoded by Label Encoder:', encoded_words)
        
        encoded_words = onehot_enc.transform(encoded_words.reshape(len(encoded_words), 1))
        print('Encoded by OneHot Encoder:')
        print(encoded_words)

        results.append(np.sum(encoded_words.toarray(), axis=0))
    
    return results

transformed_results = transform([
    news_headline1_tokens, news_headline2_tokens, news_headline3_tokens, news_headline4_tokens])


All Tokens:
['Web', 'services', 'are', 'XML', 'based', 'information', 'exchange', 'systems', 'that', 'use', 'the', 'Internet', 'for', 'direct', 'application-to-application', 'interaction', '.', 'These', 'systems', 'can', 'include', 'programs', ',', 'objects', ',', 'messages', ',', 'or', 'documents', '.', 'Web', 'services', 'are', 'XML', 'based', 'information', 'exchange', 'systems', 'that', 'use', 'the', 'Internet', 'for', 'direct', 'application-to-application', 'interaction', '.', 'These', 'systems', 'can', 'include', 'programs', ',', 'objects', ',', 'messages', ',', 'or', 'documents', '.', 'Web', 'services', 'are', 'XML', 'based', 'information', 'systems', 'and', 'use', 'Internet', 'for', 'application', 'interaction.They', 'contain', 'programs', ',', 'objects', ',', 'messages', ',', 'or', 'documents', '.', 'Web', 'services', 'Web', 'services', 'Web', 'services', 'XML', 'Web', 'services', 'XML', 'Internet']

Original Input: ['Web', 'services', 'are', 'XML', 'based', 'information', 'e

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [50]:
#EEuclidean Distance

print('Master Sentence: %s' % news_headlines[0])
for i, news_headline in enumerate(news_headlines):
    score = sklearn.metrics.pairwise.euclidean_distances([transformed_results[i]], [transformed_results[0]])[0][0]
    print('-----')
    print('Score: %.2f, Comparing Sentence: %s' % (score, news_headline))

Master Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 0.00, Comparing Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 0.00, Comparing Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 3.87, Comparing Sentence: Web services are XML based information  systems and use Internet for application interaction.They contain programs, objects, messages, or documents.
-----
Score: 7.42, Comparing Sentence: Web services Web services Web services XML Web services XML Internet 


In [51]:
#Cosine Similarity

print('Master Sentence: %s' % news_headlines[0])
for i, news_headline in enumerate(news_headlines):
    score = sklearn.metrics.pairwise.cosine_similarity([transformed_results[i]], [transformed_results[0]])[0][0]
    print('-----')
    print('Score: %.2f, Comparing Sentence: %s' % (score, news_headline))

Master Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 1.00, Comparing Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 1.00, Comparing Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 0.79, Comparing Sentence: Web services are XML based information  systems and use Internet for application interaction.They contain programs, objects, messages, or documents.
-----
Score: 0.29, Comparing Sentence: Web services Web services Web services XML Web services XML Internet 


In [52]:
#Jaccard Similarity
"""
    Finding the posistion (from lookup table) of word instead of using 1 or 0
    to prevent misleading of the meaning of "common" word
"""

def calculate_position(values):
    x = []
    for pos, matrix in enumerate(values):
        if matrix > 0:
            x.append(pos)
    return x

"""
    Since scikit-learn can only compare same number of dimension of input. 
    Add padding to the shortest sentence.
"""
def padding(sentence1, sentence2):
    x1 = sentence1.copy()
    x2 = sentence2.copy()
    
    diff = len(x1) - len(x2)
    
    if diff > 0:
        for i in range(0, diff):
            x2.append(-1)
    elif diff < 0:
        for i in range(0, abs(diff)):
            x1.append(-1)
    
    return x1, x2    

y_actual = calculate_position(transformed_results[0])

print('Master Sentence: %s' % news_headlines[0])
for i, news_headline in enumerate(news_headlines):
    y_compare = calculate_position(transformed_results[i])
    x1, x2 = padding(y_actual, y_compare)
    score = sklearn.metrics.jaccard_similarity_score(x1, x2)
    print('-----')
    print('Score: %.2f, Comparing Sentence: %s' % (score, news_headline))

Master Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 1.00, Comparing Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 1.00, Comparing Sentence: Web services are XML based information exchange systems that use the Internet for direct application-to-application interaction. These systems can include programs, objects, messages, or documents.
-----
Score: 0.19, Comparing Sentence: Web services are XML based information  systems and use Internet for application interaction.They contain programs, objects, messages, or documents.
-----
Score: 0.00, Comparing Sentence: Web services Web services Web services XML Web services XML Internet 
